In [1]:
from model import TargetModel, TargetType
from model import ShadowModel, ShadowType
from model import ReconstructModule

target = TargetModel(model_type=TargetType.MBF_LARGE_V1).load('weights/backbone/mbf_large_v1.pt').freeze().eval()
shadow = ShadowModel(model_type=ShadowType.IDIAP).set_target(target)
model = ReconstructModule(target, shadow)

Loaded the checkpoint from weights/backbone/mbf_large_v1.pt


In [2]:
from dataset import HDF5Dataset, HDF5DatasetType
from dataset import FaceDataModule
dataset = HDF5Dataset(HDF5DatasetType.LFW)
dataModule = FaceDataModule(dataset)
train_loader = dataModule.train_dataloader()
val_loader = dataModule.val_dataloader()
test_loader = dataModule.test_dataloader()

Loaded dataset from /mnt/183/a/pyshen/FR_rec/dataset/HDF5Dataset/lfw.h5


In [3]:
import pytorch_lightning as pl
trainer = pl.Trainer(max_epochs=250)
trainer.fit(model, train_loader, val_loader)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2024-03-27 11:55:33.496392: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been regi

Saving to output/20240327115536(val)/0, (224, 1792, 3), (1872, 1792, 3)    
Epoch 0:  25%|██▌       | 42/166 [00:07<00:20,  5.98it/s, v_num=0, train_loss=0.0757, train_score=0.117]

/mnt/VENV/user-conda/pyshen/miniconda3/envs/rec/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [4]:
import torch
ckpt = torch.load('weights/shadow/mbf_large_v2.ckpt')
keys = list(ckpt['state_dict'].keys())
print(keys)
# shadow_ckpt = {k[7:]: ckpt['state_dict'][k] for k in keys if 'shadow' in k}
# print(shadow_ckpt.keys())
# shadow_model.backbone.load_state_dict(shadow_ckpt, strict=False)
# torch.save(shadow_model.backbone.state_dict(), 'weights/shadow/mbf_large_v2.pt')

['backbone.layers.0.layers.0.weight', 'backbone.layers.0.layers.1.weight', 'backbone.layers.0.layers.1.bias', 'backbone.layers.0.layers.1.running_mean', 'backbone.layers.0.layers.1.running_var', 'backbone.layers.0.layers.1.num_batches_tracked', 'backbone.layers.1.layers.0.layers.0.layers.0.weight', 'backbone.layers.1.layers.0.layers.0.layers.1.weight', 'backbone.layers.1.layers.0.layers.0.layers.1.bias', 'backbone.layers.1.layers.0.layers.0.layers.1.running_mean', 'backbone.layers.1.layers.0.layers.0.layers.1.running_var', 'backbone.layers.1.layers.0.layers.0.layers.1.num_batches_tracked', 'backbone.layers.1.layers.0.layers.0.layers.2.weight', 'backbone.layers.1.layers.0.layers.1.layers.0.weight', 'backbone.layers.1.layers.0.layers.1.layers.1.weight', 'backbone.layers.1.layers.0.layers.1.layers.1.bias', 'backbone.layers.1.layers.0.layers.1.layers.1.running_mean', 'backbone.layers.1.layers.0.layers.1.layers.1.running_var', 'backbone.layers.1.layers.0.layers.1.layers.1.num_batches_tracke

: 